<a href="https://colab.research.google.com/github/boodie04/-Disease-and-Symptom-Prediction-with-Precaution-Suggestions-Transformer_T5-Mini/blob/main/T5_Mini(DiseaseAndSymptoms_Disease_precaution).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cell 1: Load and Prepare Data

In [23]:
import json

notebook_path = import json

notebook_path = "/Users/boodie/Library/CloudStorage/GoogleDrive-*/YOUR_FOLDER/YOUR_NOTEBOOK.ipynb"  # UPDATE THIS PATH

with open(notebook_path, "r") as f:
    data = json.load(f)

# Remove broken widget metadata
if "widgets" in data.get("metadata", {}):
    del data["metadata"]["widgets"]

cleaned_path = notebook_path.replace(".ipynb", "_CLEAN.ipynb")

with open(cleaned_path, "w") as f:
    json.dump(data, f, indent=2)

print("✔ Cleaned notebook created:")
print(cleaned_path)
  # UPDATE THIS PATH

with open(notebook_path, "r") as f:
    data = json.load(f)

# Remove broken widget metadata
if "widgets" in data.get("metadata", {}):
    del data["metadata"]["widgets"]

cleaned_path = notebook_path.replace(".ipynb", "_CLEAN.ipynb")

with open(cleaned_path, "w") as f:
    json.dump(data, f, indent=2)

print("✔ Cleaned notebook created:")
print(cleaned_path)


In [1]:
import pandas as pd
import os

# File paths
symptoms_path = '/Users/boodie/CODES/NLP project 2025/DataSet/DiseaseAndSymptoms.csv'
precautions_path = '/Users/boodie/CODES/NLP project 2025/DataSet/Disease precaution.csv'
training_csv = '/Users/boodie/CODES/NLP project 2025/DataSet/t5_training_data.csv'

# Load CSVs
df_symptoms = pd.read_csv(symptoms_path)
df_precaution = pd.read_csv(precautions_path)

print("Symptoms Shape:", df_symptoms.shape)
print("Precautions Shape:", df_precaution.shape)

# Clean symptom columns
for col in df_symptoms.columns:
    if "Symptom" in col:
        df_symptoms[col] = df_symptoms[col].fillna('').str.replace('_', ' ').str.strip()

# Merge datasets
df = df_symptoms.merge(df_precaution, on="Disease", how="inner")
print("Merged Shape:", df.shape)

# Combine symptom columns
def combine_symptoms(row):
    symptoms = [str(row[c]) for c in row.index if "Symptom" in c and pd.notna(row[c])]
    return ", ".join(symptoms)

df["all_symptoms"] = df.apply(combine_symptoms, axis=1)

# Create encoder-decoder text
df["input_text"] = "symptoms: " + df["all_symptoms"]
df["target_text"] = df.apply(
    lambda row: f"disease: {row['Disease']} | precautions: {row['Precaution_1']}, {row['Precaution_2']}, {row['Precaution_3']}, {row['Precaution_4']}",
    axis=1
)

# Save final dataset
df_final = df[["input_text", "target_text"]]
df_final.to_csv(training_csv, index=False)
print("Final training CSV saved:", training_csv)


Symptoms Shape: (4920, 18)
Precautions Shape: (41, 5)
Merged Shape: (4920, 22)
Final training CSV saved: /Users/boodie/CODES/NLP project 2025/DataSet/t5_training_data.csv


Cell 2: Load Tokenizer & Model

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "t5-small"

tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Detect device (MPS for M1, fallback to CPU)
import torch
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print("Using device:", device)

# Move model to MPS/CPU
model.to(device)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Using device: mps


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

Cell 3: Load Dataset and Preprocess

In [3]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files=training_csv)

def preprocess(batch):
    model_inputs = tokenizer(batch["input_text"], truncation=True, padding="max_length", max_length=128)
    labels = tokenizer(text_target=batch["target_text"], truncation=True, padding="max_length", max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Map preprocessing
tokenized_data = dataset["train"].map(preprocess, batched=True, remove_columns=dataset["train"].column_names)



Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/4920 [00:00<?, ? examples/s]

Subset of the data

In [5]:
small_dataset = tokenized_data.select(range(1000))
print("Subset dataset length:", len(small_dataset))


Subset dataset length: 1000


Cell 4: Training on M1

In [6]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./t5-disease-precaution-model",
    per_device_train_batch_size=4,  # smaller for M1
    num_train_epochs=5,
    learning_rate=3e-4,
    optim="adamw_torch",
    logging_steps=50,
    save_steps=100,
    report_to="none",
    fp16=False  # M1 does not support fp16
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_dataset
)

trainer.train()


Step,Training Loss
50,0.056500
100,0.040200
150,0.031400
200,0.026900
250,0.020900
300,0.016200
350,0.011000
400,0.010400
450,0.009400
500,0.008300


/Users/boodie/miniforge3/envs/tf-metal/lib/python3.9/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/boodie/miniforge3/envs/tf-metal/lib/python3.9/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/boodie/miniforge3/envs/tf-metal/lib/python3.9/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/boodie/miniforge3/envs/tf-metal/lib/python3.9/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=1250, training_loss=0.011886989510059356, metrics={'train_runtime': 837.1292, 'train_samples_per_second': 5.973, 'train_steps_per_second': 1.493, 'total_flos': 169177251840000.0, 'train_loss': 0.011886989510059356, 'epoch': 5.0})

Cell 5: Save Model & Tokenizer

In [7]:
model.cpu()  # move to CPU before saving
model.save_pretrained("./t5-disease-precaution-model")
tokenizer.save_pretrained("./t5-disease-precaution-model")
print("Model saved locally.")


Model saved locally.


In [9]:
import os

saved_path = "./t5-disease-precaution-model"
print(os.listdir(saved_path))


['model.safetensors', 'added_tokens.json', 'checkpoint-500', 'tokenizer_config.json', 'special_tokens_map.json', 'checkpoint-100', 'checkpoint-1250', 'checkpoint-1000', 'config.json', 'checkpoint-600', 'checkpoint-800', 'generation_config.json', 'checkpoint-200', 'spiece.model', 'checkpoint-700', 'checkpoint-1200', 'checkpoint-300', 'checkpoint-900', 'checkpoint-400', 'checkpoint-1100']


Cell 6: Inference / Prediction

In [10]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

tokenizer = T5Tokenizer.from_pretrained(saved_path)
model = T5ForConditionalGeneration.from_pretrained(saved_path)
model.to(device)  # IMPORTANT: move to MPS or CPU
model.eval()      # set to evaluation mode


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [11]:
def predict_precaution(symptoms_list):
    input_text = "symptoms: " + ", ".join(symptoms_list)

    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    ).to(device)  # move inputs to same device as model

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=128, num_beams=5)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [13]:
symptoms_example = ["fever", "cough", "headache"]
result = predict_precaution(symptoms_example)
print("Predicted Output:", result)


Predicted Output: disease: Malaria | precautions: Consult nearest hospital, avoid oily food, avoid non veg food, keep mosquitos out


Cell 7: Evaluate Model

In [19]:
from torch.utils.data import DataLoader
from nltk.translate.bleu_score import sentence_bleu
import torch

# Make sure device is set
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
model.to(device)
model.eval()

def evaluate_model(model, tokenizer, dataset, batch_size=4, max_length=128):
    predictions = []
    references = []

    # --- Tokenize dataset ---
    def preprocess_eval(batch):
        inputs = tokenizer(batch["input_text"], truncation=True, padding="max_length", max_length=max_length)
        labels = tokenizer(text_target=batch["target_text"], truncation=True, padding="max_length", max_length=max_length)
        inputs["labels"] = labels["input_ids"]
        return inputs

    tokenized_test = dataset.map(preprocess_eval, batched=True, remove_columns=dataset.column_names)
    tokenized_test.set_format(type="torch")  # <-- convert to PyTorch tensors

    loader = DataLoader(tokenized_test, batch_size=batch_size)

    for batch in loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=max_length)

        preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        labels_decoded = tokenizer.batch_decode(labels, skip_special_tokens=True)

        predictions.extend(preds)
        references.extend(labels_decoded)

    # --- Metrics ---
    exact_matches = sum([p == r for p, r in zip(predictions, references)])
    exact_accuracy = exact_matches / len(references)

    bleu_scores = []
    for p, r in zip(predictions, references):
        if len(p.strip()) == 0 or len(r.strip()) == 0:
            bleu_scores.append(0.0)
        else:
            bleu_scores.append(sentence_bleu([r.split()], p.split(), weights=(0.5, 0.5)))

    avg_bleu = sum(bleu_scores) / len(bleu_scores)

    print(f"Exact Match Accuracy: {exact_accuracy:.4f}")
    print(f"Average BLEU Score : {avg_bleu:.4f}")

    return predictions, references

# --- Run Evaluation ---
preds, refs = evaluate_model(
    model,
    tokenizer,
    dataset["train"].select(range(1000)),
    batch_size=16,
    max_length=64
)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Exact Match Accuracy: 0.9980
Average BLEU Score : 0.9997
